In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd 

In [2]:
def dense_to_one_hot(label_dense,num_classes):
    num_lables = label_dense.shape[0]
    index_offset = np.arange(num_lables) *num_classes
    label_one_hot = np.zeros((num_lables, num_classes))
    label_one_hot.flat[index_offset + label_dense.ravel()] = 1
    return label_one_hot

#### 读取数据

In [3]:
train = pd.read_csv("data/train.csv")
images = train.iloc[:,1:].values
labels_flat = train.iloc[:,:1].values.ravel()

images = images.astype(np.float)
images = np.multiply(images, 1.0/255.0)
image_size = images.shape[1]
image_width = image_height = np.ceil(np.sqrt(image_size)).astype(np.uint8)
x = tf.placeholder("float", shape=[None, image_size])
labels_count = np.unique(labels_flat).shape[0]
y = tf.placeholder("float", shape=[None, labels_count])
labels = dense_to_one_hot(labels_flat,labels_count)
labels = labels.astype(np.uint8)

#### 数据分集

In [4]:
VALIDATION_SIZE = 2000
validation_images = images[:VALIDATION_SIZE]
validation_labels = labels[:VALIDATION_SIZE]
train_images = images[VALIDATION_SIZE:]
train_labels = labels[VALIDATION_SIZE:]

#### 模型定义

In [5]:
def weight_varible(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_varible(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x,w):
    return tf.nn.conv2d(x,w,strides=[1,1,1,1],padding="VALID")

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")

####  模型

In [6]:
x_image = tf.reshape(x,[-1,28,28,1])

# （28-3 + 1）/ 2 = 13
w_conv1 = weight_varible([3,3,1,16])
b_conv1 = bias_varible([16])
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

#  (13-6+1)/2 = 4 
w_conv2 = weight_varible([6,6,16,128])
b_conv2 = bias_varible([128])
h_conv2 = tf.nn.relu(conv2d(h_pool1,w_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
h_pool2_flat = tf.reshape(h_pool2,[-1,4*4*128])

w_fc1 = weight_varible([4*4*128,128])
b_fc1 = bias_varible([128])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,w_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

w_fc2 = weight_varible([128,10])
b_fc2 = bias_varible([10])
y_conv = tf.matmul(h_fc1_drop,w_fc2)+b_fc2

#### 定义损失函数与优化方式

In [7]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=y_conv))

train_step = tf.train.AdadeltaOptimizer(learning_rate=0.1).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_conv,1))

accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

global_step = tf.Variable(0,name='global_step',trainable=False)

saver= tf.train.Saver()

init = tf.global_variables_initializer()

#### 进行训练

In [8]:
with tf.Session() as sess:
    sess.run(init)
    
    for enpoch in range(20):
        batch_size = 100
        n_batch = len(train_images)/batch_size
        for batch in range(n_batch):
#             print("----------->round:"+str(enpoch+1) + "/5, rate:"+str(batch+1) + "/" + str(n_batch))
            batch_x = train_images[batch*batch_size:(batch+1)*batch_size]
            batch_y = train_labels[batch*batch_size:(batch+1)*batch_size]
            sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:0.5})
            if batch==n_batch-1:
                sess.run(train_step,feed_dict={x:batch_x,y:batch_y,keep_prob:0.5})
                accuracy_n = sess.run(accuracy,feed_dict={x:batch_x,y:batch_y,keep_prob:1.0})
                print("第"+str(enpoch+1)+"轮，训练准确度为："+str(accuracy_n))
            
        accuracy_n = sess.run(accuracy,feed_dict={x:validation_images,y:validation_labels,keep_prob:1.0})
        print("第"+str(enpoch+1)+"轮，准确度为："+str(accuracy_n))
        global_step.assign(enpoch).eval()
        saver.save(sess,"/Users/max/Desktop/TensorFlow/tmp/model.ckpt",global_step=global_step)

第1轮，训练准确度为：0.8
第1轮，准确度为：0.7945
第2轮，训练准确度为：0.91
第2轮，准确度为：0.881
第3轮，训练准确度为：0.93
第3轮，准确度为：0.8995
第4轮，训练准确度为：0.94
第4轮，准确度为：0.9175
第5轮，训练准确度为：0.94
第5轮，准确度为：0.929
第6轮，训练准确度为：0.96
第6轮，准确度为：0.937
第7轮，训练准确度为：0.97
第7轮，准确度为：0.9415
第8轮，训练准确度为：0.97
第8轮，准确度为：0.945
第9轮，训练准确度为：0.97
第9轮，准确度为：0.949
第10轮，训练准确度为：0.97
第10轮，准确度为：0.9535
第11轮，训练准确度为：0.97
第11轮，准确度为：0.957
第12轮，训练准确度为：0.97
第12轮，准确度为：0.9595
第13轮，训练准确度为：0.97
第13轮，准确度为：0.9645
第14轮，训练准确度为：0.97
第14轮，准确度为：0.9665
第15轮，训练准确度为：0.98
第15轮，准确度为：0.97
第16轮，训练准确度为：0.98
第16轮，准确度为：0.9705
第17轮，训练准确度为：0.98
第17轮，准确度为：0.974
第18轮，训练准确度为：0.99
第18轮，准确度为：0.9735
第19轮，训练准确度为：0.98
第19轮，准确度为：0.9745
第20轮，训练准确度为：0.99
第20轮，准确度为：0.975
